<h1>Description of Networks by Characteristics</h1>
This notebook describes the different networks in the article, as well as the networks created as negatives using Rewarding and Erdos Renyi.

In [1]:
import networkx as nx
import pandas as pd
import random

In [2]:
n_file=["carnival_tourist","no_carnival_tourist","carnival_residents","no_carnival_residents"]
list_Graphs_names=list()
for i in n_file:
    list_Graphs_names.append('../Datos/'+i+'.graphml')


<h2>Implementation of the Rewiring algorithm</h2>

In [3]:
def rewiring(G_original): 
    """
    Using the original network, redistribute the links and weights
    :param G_original: networkX DiGraph 
    :type G_original: Graph
    :return: new rewiring graph with weights
    :rtype: DiGraph
    """
    #Obtain original nodes and weights
    nodos = list(G_original.nodes())
    pesos = [G_original[u][v]['weight'] for u, v in G_original.edges()]
    num_aristas = G_original.number_of_edges()
    
    #Create new network with same nodes
    G_random = nx.DiGraph()  # o nx.Graph() si no dirigida
    G_random.add_nodes_from(nodos)
    
    #Generate random edges without repetition 
    posibles_aristas = [(u, v) for u in nodos for v in nodos if u != v]
    aristas_aleatorias = random.sample(posibles_aristas, num_aristas)
    
    #Mix the weights 
    random.shuffle(pesos)
    
    #Assign weights to random edges
    for (u, v), peso in zip(aristas_aleatorias, pesos):
        G_random.add_edge(u, v, weight=peso)
    return G_random

<h2>Implementation of the Erdos Renyi</h2>

In [4]:
def erdos_renyi_weighted_same_nodes(G_original, weight_range=(0.1, 1.0), directed=True, seed=None):
    """
    Generates an Erdős-Rényi network with the same nodes and number of edges as G_original, and assigns random weights to the edges.
    :param G_original: networkX DiGraph.
    :type G_original: DiGraph
    :param weight_range: tuple with the range of weights (min, max)
    :type weight_range: tuple
    :param directed: if True, generates a directed graph
    :type directed: bool
    :param seed: optional random seed
    :type seed: int
    :return: new random graph with weights
    :rtype: DiGraph
    """
    
    rng = random.Random(seed)
    n = G_original.number_of_nodes()
    m = G_original.number_of_edges()
    
    # Create an empty graph with the same nodes
    G_rand = nx.DiGraph() if directed else nx.Graph()
    G_rand.add_nodes_from(G_original.nodes())

    possible_edges = [(u, v) for u in G_rand.nodes() for v in G_rand.nodes() if u != v]
    if not directed:
        possible_edges = [(u, v) for u, v in possible_edges if u < v]

    rng.shuffle(possible_edges)
    selected_edges = possible_edges[:m]

    for u, v in selected_edges:
        weight = rng.uniform(*weight_range)
        G_rand.add_edge(u, v, weight=weight)

    return G_rand

<h2>Characteristic calculation function</h2>

In [5]:
def description(list_Graphs,names):
    dict_description=dict()
    for c,G in enumerate(list_Graphs):
        i=names[c]
        dict_description[i]=list()
        dict_description[i].append(sum([d for n, d in G.degree(weight="weight")])/len([d for n, d in G.in_degree()]))      
        #The diameter and SPL require a strongly connected graph.    
        if not nx.is_strongly_connected(G):
            # Obtain the largest strongly connected component
            largest_scc = max(nx.strongly_connected_components(G), key=len)
            # Create subgraph with that component
            G = G.subgraph(largest_scc).copy()
        dict_description[i].append(nx.average_shortest_path_length(G,weight="weight"))
        dict_description[i].append(nx.diameter(G,weight="weight"))
    return dict_description


<h2>Characteristics of real networks</h2>

In [6]:
list_Graphs=list()
for i in list_Graphs_names:
    list_Graphs.append(nx.read_graphml(i))
dict_description=description(list_Graphs,n_file)
pd.DataFrame(dict_description,index=["degree_mean","avg_spl","diameter"])

,carnival_tourist,no_carnival_tourist,carnival_residents,no_carnival_residents
degree_mean,2015.555556,3267.093750,641.965517,1472.063492
avg_spl,3.473684,3.589617,2.519985,2.601110
diameter,10.000000,11.000000,9.000000,8.000000


<h2>Characteristics of Rewiring networks</h2>

In [7]:
list_Graphs=list()
for i in list_Graphs_names:
    list_Graphs.append(rewiring(nx.read_graphml(i)))
dict_description=description(list_Graphs,n_file)
pd.DataFrame(dict_description,index=["degree_mean","avg_spl","diameter"])


,carnival_tourist,no_carnival_tourist,carnival_residents,no_carnival_residents
degree_mean,2015.555556,3267.093750,641.965517,1472.063492
avg_spl,3.479519,3.517857,2.516636,2.406810
diameter,7.000000,7.000000,5.000000,4.000000


<h2>Characteristics of Erdos Renyi networks</h2>

In [8]:
list_Graphs=list()
for i in list_Graphs_names:
    G=nx.read_graphml(i)
    pesos = [d['weight'] for u, v, d in G.edges(data=True) if 'weight' in d]
    # Obtener mínimo y máximo
    peso_min = min(pesos)
    peso_max = max(pesos)
    list_Graphs.append(rewiring(erdos_renyi_weighted_same_nodes(G, weight_range=(peso_min, peso_max), directed=True, seed=None)))
dict_description=description(list_Graphs,n_file)
pd.DataFrame(dict_description,index=["degree_mean","avg_spl","diameter"])



,carnival_tourist,no_carnival_tourist,carnival_residents,no_carnival_residents
degree_mean,61833.398789,125083.063138,40179.997308,135184.869976
avg_spl,445.501299,656.423032,664.745761,1294.045455
diameter,1537.933716,1845.483821,1759.315905,3940.531561
